# 2. Pytorch workflow


In [ ]:
import torch as tc
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

## 2.1 Preparing and Loading Data


In [ ]:
tc.__version__

In [ ]:
# create known params
weight = 0.7
bias = 0.3

# create numbers
start = 0
end = 1
step = 0.02
X = tc.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias
X[:10], y[:10]

### 1.1.1 Data splitting


In [ ]:
# create test split
# create test split
# from sklearn.model_selection import train_test_split

# train_split = int(0.8 * len(X))
# X_train, y_train, X_test, y_test = train_test_split(X, test_size=0.2, random_state=101)

# print(X_train, "\n", X_test, "\n", y_train, "\n", y_test)

train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]
# print(X_train[:10], "\n", X_test[:5], "\n", y_train[:10], "\n", y_test[:5])
len(X_train), len(y_train), len(X_test), len(y_test)

## 2.2 Visualization


In [ ]:
def plot_pred(
    train_data=X_train,
    train_label=y_train,
    test_data=X_test,
    test_label=y_test,
    predictions=None,
):
    """
    Function to train data, test data and compare predictions
    """
    plt.figure(figsize=(10, 7))
    plt.scatter(train_data, train_label, c="b", s=16, label="Training data")
    plt.scatter(test_data, test_label, c="g", s=16, label="Testing data")

    # Are there predictions?
    if predictions is not None:
        plt.scatter(test_data, predictions, c="r", label="Predictions")

    plt.legend(prop={"size": 14})

In [ ]:
plot_pred()

## 2.3 Building a model


In [ ]:
# creating a linear regression model
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(tc.randn(1, requires_grad=True, dtype=tc.float))
        self.bias = nn.Parameter(tc.randn(1, requires_grad=True, dtype=tc.float))

        # forward methodez
    def forward(self, x: tc.tensor) -> tc.Tensor:
        return self.weights * x + self.bias

In [ ]:
# creating a random seed
tc.manual_seed(42)

model0 = LinearRegressionModel()
model0

In [ ]:
# check the parameters
list(model0.parameters())

## 2.4 Making predictions from our model

In [ ]:
with tc.inference_mode():
  y_preds = model0(X_test)
y_preds

In [ ]:
plot_pred(predictions=y_preds)

## 2.5 Training a model

In [ ]:
list(model0.parameters())

In [ ]:
print(model0.state_dict())
loss_fn = nn.L1Loss()
print(loss_fn)

In [ ]:
# setting up optimiser
optimiser = tc.optim.SGD(params=model0.parameters(),lr=0.01)
print(optimiser)

In [ ]:
with tc.no_grad():
  list(model0.parameters())

In [ ]:
# building a training loop
tc.manual_seed(42)
epochs = 200

# TRacking the values
epoch_count = []
loss_values = []
test_loss_values = []

# loop through the data
for epoch in range(epochs):
  # set the model to training mode
  model0.train() # train mode in pytorch sets all parameters that require gradient to train
  
  # 1. forward pass
  y_pred =  model0(X_train)
  
  # 2. Calculate the loss
  loss = loss_fn(y_pred,y_train)
  # print(f"Loss: {loss}")
  
  # 3. Optimise zero grad
  optimiser.zero_grad()
  
  # 4. Perform backpropagation on the loss with respect to the parameters of the model
  loss.backward()
  
  # 5. Step the optimiser (gradient descent)
  optimiser.step() 
  model0.eval() # turns off different meetings in the model not needed for evaluation
  with tc.inference_mode(): # turns off gradient tracking
    # 1. Do the forward pass
    test_pred = model0(X_test)
    
    # 2. Calculate the loss
    test_loss = loss_fn(test_pred,y_test)
  if epoch % 10 == 0:
    epoch_count.append(epoch)
    loss_values.append(loss)
    test_loss_values.append(test_loss)
    print(f"Epoch: {epoch} | Loss: {loss} | Test loss: {test_loss}")
    # print out model state_dict()
    print(model0.state_dict())

In [ ]:
print(model0.state_dict())

In [ ]:
with tc.inference_mode():
  y_preds_new = model0(X_test)
  
y_preds_new

In [ ]:
plot_pred(predictions=y_preds)

In [ ]:
plot_pred(predictions=y_preds_new)

In [ ]:
# plot the loss curves
plt.plot(epoch_count,np.array(tc.tensor(loss_values).cpu().numpy()),label="Train Loss")
plt.plot(epoch_count,test_loss_values,label="Test loss")
plt.title("Training and test loss curve")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend()

## 2.6 Saving a model

In [ ]:
# saving our pytorch model
from pathlib import Path
# create model directory
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True,exist_ok=True)
MODEL_NAME = "01_pytorch_workflow_model.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME
MODEL_SAVE_PATH

# save the model state_dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
tc.save(obj=model0.state_dict(),f=MODEL_SAVE_PATH)


In [ ]:
# Loading a model
loaded_model0 = LinearRegressionModel()

# load the saved state_dict of model 0
loaded_model0.load_state_dict(tc.load(f=MODEL_SAVE_PATH))

In [ ]:
loaded_model0.eval()
with tc.inference_mode():
  loaded_model_preds = loaded_model0(X_test)
  
loaded_model_preds


In [ ]:
# compare the two models
y_preds_new == loaded_model_preds